In [ ]:
# Import and Initialize bedrock client
import os
import boto3
from botocore.exceptions import ClientError
from common.common import stream_conversation, parser, save_files
from common.gherkin_common import get_gherkin_prompt_message
from common.gherkin_common import input_api_spec_file_dropdown, \
    input_api_spec_path_dropdown, input_api_spec_method_dropdown, \
    input_api_spec_response_code_dropdown, refresh_input
from IPython.display import display
from common.gherkin_stub_common import input_feature_dropdown, \
    get_java_stub_prompt_message, refresh_features

model_id = os.getenv("model_id")
# inference parameters to use.
temperature = 0
top_k = 250
top_p = 1
# Base inference parameters.
inference_config = {
    "temperature": temperature,
    "topP": top_p
}
# Additional model inference parameters.
additional_model_fields = {"top_k": top_k}

bedrock_client = boto3.client(service_name='bedrock-runtime')
print(model_id)

In [ ]:
# Show interactive display to select the path, method and response code
refresh_input()
display(input_api_spec_file_dropdown)
display(input_api_spec_path_dropdown)
display(input_api_spec_method_dropdown)
display(input_api_spec_response_code_dropdown)

In [ ]:
# Generate the feature file.
system_prompts = []
feature_file_name, path, operation, response_code, messages = \
    get_gherkin_prompt_message()

try:
    bedrock_client = boto3.client(service_name='bedrock-runtime')
    result = stream_conversation(bedrock_client, model_id, messages,
                                 system_prompts, inference_config,
                                 additional_model_fields)
    result = parser(result, "gherkin_scenarios")
    feature_file_name = (
        f"{feature_file_name}_{path[1:].replace('/', '_')}_"
        f"{operation}_{response_code}.feature"
    )
    print(feature_file_name)
    save_files(content=result, root="output/features",
               file_name=feature_file_name)
except ClientError as err:
    message = err.response['Error']['Message']
    print("A client error occurred: " + format(message))
else:
    print(f"Finished streaming messages with model {model_id}.")

In [ ]:
# Show interactive display to select the feature files.
refresh_features()
display(input_feature_dropdown)

In [ ]:
# Generate the java step definitions file.

feature_file_name, messages = get_java_stub_prompt_message()

try:
    result = stream_conversation(bedrock_client, model_id, messages,
                                 system_prompts, inference_config,
                                 additional_model_fields)
    result = parser(result, "java_code")
    feature_file_name = f"{feature_file_name}.java"
    save_files(result, "output/stubs", feature_file_name)
except ClientError as err:
    message = err.response['Error']['Message']
    print("A client error occurred: " + format(message))
else:
    print(f"Finished streaming messages with model {model_id}.")